Note, that you will obtain a error on test set :)

In [1]:
%load_ext autoreload

%autoreload 2


import warnings


warnings.filterwarnings('ignore')

### Load tfidf model

In [2]:
import pickle
import codecs
import joblib

In [3]:
vect = joblib.load("ruentfidf/tfidf.pkl")

### Load data

In [ ]:
!wget https://russiansuperglue.com/tasks/download
!unzip download
!rm download

In [6]:
from pathlib import Path

In [7]:
data_dir = Path("combined/")

In [8]:
all_results = {
    "name": [],
    "train": [],
    "val": [],
    "test": []
}

### PARus

1. build text: "{premise} {question} {choice1} {choice2}"
2. get tfidf
3. fit logreg


In [9]:
train_path = data_dir / "PARus/train.jsonl"
val_path = data_dir / "PARus/val.jsonl"
test_path = data_dir / "PARus/test.jsonl"

In [10]:
from tfidf_baseline.PARus import eval_PARus

In [11]:
_, PARus_scores = eval_PARus(train_path, val_path, test_path, vect)

In [12]:
PARus_scores

{'train': 0.775, 'val': 0.45, 'test': 0.486}

In [13]:
all_results["name"].append("PARus")
all_results["train"].append(PARus_scores["train"])
all_results["val"].append(PARus_scores["val"])
all_results["test"].append(PARus_scores["test"])

### RCB
1. build text: "{premise} {hypothesis}"
2. get tfidf
3. fit logreg

In [14]:
train_path = data_dir / "RCB/train.jsonl"
val_path = data_dir / "RCB/val.jsonl"
test_path = data_dir / "RCB/test.jsonl"

In [15]:
from tfidf_baseline.RCB import eval_RCB

In [16]:
_, RCB_scores = eval_RCB(train_path, val_path, test_path, vect)

In [17]:
RCB_scores

{'train': 0.7420091324200914,
 'val': 0.5227272727272727,
 'test': 0.4520547945205479}

In [18]:
all_results["name"].append("RCB")
all_results["train"].append(RCB_scores["train"])
all_results["val"].append(RCB_scores["val"])
all_results["test"].append(RCB_scores["test"])

### DaNetQA
1. build text: "{question}"
2. get tfidf
3. fit logreg

In [19]:
train_path = data_dir / "DaNetQA/train.jsonl"
val_path = data_dir / "DaNetQA/val.jsonl"
test_path = data_dir / "DaNetQA/test.jsonl"

In [20]:
from tfidf_baseline.DaNetQA import eval_DaNetQA

In [21]:
_, DaNetQA_scores = eval_DaNetQA(train_path, val_path, test_path, vect)

In [22]:
DaNetQA_scores

{'train': 0.7321428571428571,
 'val': 0.6644067796610169,
 'test': 0.6847457627118644}

In [23]:
all_results["name"].append("DaNetQA")
all_results["train"].append(DaNetQA_scores["train"])
all_results["val"].append(DaNetQA_scores["val"])
all_results["test"].append(DaNetQA_scores["test"])

### TERRa
1. build text: "{premise} {hypothesis}"
2. get tfidf
3. fit logreg


In [24]:
train_path = data_dir / "TERRA/train.jsonl"
val_path = data_dir / "TERRA/val.jsonl"
test_path = data_dir / "TERRA/test.jsonl"

In [25]:
from tfidf_baseline.TERRa import eval_TERRa

In [26]:
_, TERRa_scores = eval_TERRa(train_path, val_path, test_path, vect)

In [27]:
TERRa_scores

{'train': 0.7152140672782875,
 'val': 0.46579804560260585,
 'test': 0.4715447154471545}

In [28]:
all_results["name"].append("TERRa")
all_results["train"].append(TERRa_scores["train"])
all_results["val"].append(TERRa_scores["val"])
all_results["test"].append(TERRa_scores["test"])

### RWSD
1. build text: "{premise} {span1} {span2}"
2. get tfidf
3. fit logreg


In [29]:
train_path = data_dir / "RWSD/train.jsonl"
val_path = data_dir / "RWSD/val.jsonl"
test_path = data_dir / "RWSD/test.jsonl"

In [30]:
from tfidf_baseline.RWSD import eval_RWSD

In [31]:
_, RWSD_scores = eval_RWSD(train_path, val_path, test_path, vect)

In [32]:
RWSD_scores

{'train': 0.5115511551155115,
 'val': 0.553921568627451,
 'test': 0.6623376623376623}

In [33]:
all_results["name"].append("RWSD")
all_results["train"].append(RWSD_scores["train"])
all_results["val"].append(RWSD_scores["val"])
all_results["test"].append(RWSD_scores["test"])

### RUSSE
1. build text: "{sentence1} {sentence2} {word}"
2. get tfidf
3. fit logreg

In [34]:
train_path = data_dir / "RUSSE/train.jsonl"
val_path = data_dir / "RUSSE/val.jsonl"
test_path = data_dir / "RUSSE/test.jsonl"

In [35]:
from tfidf_baseline.RUSSE import eval_RUSSE

In [36]:
_, RUSSE_scores = eval_RUSSE(train_path, val_path, test_path, vect)

In [37]:
RUSSE_scores

{'train': 0.7103552532123961,
 'val': 0.6653733098177542,
 'test': 0.6694922228623159}

In [38]:
all_results["name"].append("RUSSE")
all_results["train"].append(RUSSE_scores["train"])
all_results["val"].append(RUSSE_scores["val"])
all_results["test"].append(RUSSE_scores["test"])

### LiDiRus
1. build text: "{sentence1} {sentence2}"
2. get tfidf
3. fit logreg

In [39]:
train_path = data_dir / "TERRA/train.jsonl"
val_path = data_dir / "TERRA/val.jsonl"
test_path = data_dir / "LiDiRus/LiDiRuS.jsonl"

In [40]:
from tfidf_baseline.LiDiRus import eval_LiDiRus

In [41]:
_, LiDiRus_scores = eval_LiDiRus(train_path, val_path, test_path, vect)

In [42]:
LiDiRus_scores

{'train': 0.4294719661883857,
 'val': -0.06835232958984723,
 'test': 0.05974021843803689}

In [43]:
all_results["name"].append("LiDiRus")
all_results["train"].append(LiDiRus_scores["train"])
all_results["val"].append(LiDiRus_scores["val"])
all_results["test"].append(LiDiRus_scores["test"])

### RuCoS
1. build text of passage and queries
2. get tfidf of passage and queries
3. calculate cosins between passage and queries
4. select best by cosin metric

In [44]:
train_path = data_dir / "RuCoS/train.jsonl"
val_path = data_dir / "RuCoS/dev.jsonl"
test_path = data_dir / "RuCoS/test.jsonl"

In [45]:
from tfidf_baseline.RuCoS import eval_RuCoS

In [46]:
_, RuCoS_scores = eval_RuCoS(train_path, val_path, test_path, vect)

In [47]:
RuCoS_scores

{'train': (0.20824733699943207, 0.2263773335525391),
 'val': (0.22951036030091065, 0.23539879459768612),
 'test': (0.25151598676957, 0.2564189898671709)}

In [48]:
all_results["name"].append("RuCoS")
all_results["train"].append(RuCoS_scores["train"])
all_results["val"].append(RuCoS_scores["val"])
all_results["test"].append(RuCoS_scores["test"])

### MuSeRC
1. build text of passage and queries
2. get tfidf of passage and queries
3. calculate cosins between passage and queries
4. select best 2 by cosin metric

In [55]:
train_path = data_dir / "MuSeRC/train.jsonl"
val_path = data_dir / "MuSeRC/val.jsonl"
test_path = data_dir / "MuSeRC/test.jsonl"

In [56]:
from tfidf_baseline.MuSeRC import eval_MuSeRC

In [57]:
_, MuSeRC_scores = eval_MuSeRC(train_path, val_path, test_path, vect)

In [58]:
MuSeRC_scores

{'train': (0.2140077821011673, 0.5475732090384031),
 'val': (0.20982986767485823, 0.5207215992198928),
 'test': (0.24434638720353005, 0.5895127875410773)}

In [59]:
all_results["name"].append("MuSeRC")
all_results["train"].append(MuSeRC_scores["train"])
all_results["val"].append(MuSeRC_scores["val"])
all_results["test"].append(MuSeRC_scores["test"])

### Over All

In [68]:
import pandas as pd

In [69]:
results = pd.DataFrame(all_results)

In [70]:
results

,name,train,val,test
0,PARus,0.775,0.45,0.486
1,RCB,0.742009,0.522727,0.452055
2,DaNetQA,0.732143,0.664407,0.684746
3,TERRa,0.715214,0.465798,0.471545
4,RWSD,0.511551,0.553922,0.662338
5,RUSSE,0.710355,0.665373,0.669492
6,LiDiRus,0.429472,-0.0683523,0.0597402
7,RuCoS,"(0.20824733699943207, 0.2263773335525391)","(0.22951036030091065, 0.23539879459768612)","(0.25151598676957, 0.2564189898671709)"
8,MuSeRC,"(0.2140077821011673, 0.5475732090384031)","(0.20982986767485823, 0.5207215992198928)","(0.24434638720353005, 0.5895127875410773)"


In [71]:
results.to_csv("results.csv", sep="\t")